Creacion de dataset, haciendo scrapping de distintos productos de amazon para extraer las reseñas junto con su etiqueta(numero de estrellas)

In [ ]:
import requests
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from threading import Thread, Lock, Semaphore

# Lista de productos a buscar
productos = [
    "Audífonos inalámbricos", "Cargador portátil", "Protector de pantalla", "Fundas para celular", "Cable HDMI",
    "Tarjeta de memoria", "Adaptador USB", "Smartwatch", "Cámara de seguridad", "Luz LED",
    "Smart TV", "Mouse inalámbrico", "Lector de tarjetas", "Cable auxiliar", "Bocina Bluetooth",
    "Cámara web", "Mini proyector", "Teclado inalámbrico", "Adaptador de corriente", "Cargador rápido",
    "Juego de sartenes", "Hervidor de agua", "Dispensador de jabón", "Desinfectante", "Toallas de papel",
    "Parrilla eléctrica", "Reloj despertador", "Recipiente de almacenamiento", "Cafetera de cápsulas", "Tetera",
    "Olla de presión", "Batidora de mano", "Procesador de alimentos", "Juego de cuchillos", "Aspiradora portátil",
    "Plancha para el cabello", "Secadora de cabello", "Depiladora", "Cortadora de cabello", "Cepillo de dientes eléctrico",
    "Humidificador de aire", "Difusor de aromas", "Filtro de agua", "Ventilador de torre", "Calefactor",
    "Manta eléctrica", "Almohada ortopédica", "Cubrecolchón", "Sábanas", "Perchero",
    "Mesa plegable", "Taburete", "Repisa flotante", "Ganchos adhesivos", "Caja organizadora",
    "Candelabro", "Lámpara de escritorio", "Espejo de aumento", "Estantería", "Tapete de yoga",
    "Reloj de pared", "Plafón LED", "Alfombra", "Vaporizador de ropa", "Cajonera",
    "Estante de cocina", "Tendedero", "Tabla de picar", "Cacerola", "Porta utensilios",
    "Bolsas de basura", "Dispensador de agua", "Refrigerador portátil", "Parrilla para asar", "Guantes de cocina",
    "Organizador de zapatos", "Cuerda para saltar", "Mancuernas ajustables", "Rodillo de espuma", "Soga de resistencia",
    "Guantes de entrenamiento", "Maletín para laptop", "Protector solar", "Mascarillas faciales", "Suero facial",
    "Cinturón de ejercicio", "Báscula de baño", "Banda de resistencia", "Colgador de pared", "Cinta métrica",
    "Pulsera de actividad", "Guantes de ciclismo", "Herramienta múltiple", "Linterna recargable", "Cámara de acción",
    "Cuaderno inteligente", "Funda para laptop", "Enfriador de laptop", "Bomba de agua portátil", "Llavero organizador",
    "Funda de tablet", "Cortina de baño", "Barra de cortina", "Cortadora de verduras", "Juego de tazas medidoras"
]


# URL base de Amazon México
url = "https://www.amazon.com.mx/s"

# Encabezados personalizados
custom_headers = {
    'user-agent': generate_user_agent(),
    'accept-language': 'en-GB,en;q=0.9',
}

# Numero de enlaces de productos a extraer para cada tipo de producto
num_products = 20
# Configuración de paralelismo
num_cores = 6
chunk_size = len(productos) // num_cores

# Exclusión mutua para el acceso seguro al DataFrame
data_lock = Lock()

# DataFrame para almacenar las reseñas
all_reviews = pd.DataFrame(columns=["Producto", "Reseña", "Calificación"])

# Función para extraer enlaces de productos
def get_product_links(product_name):
    product_links_list = []
    params = {'k': product_name}
    current_url = url

    while len(product_links_list) < num_products:
        response = requests.get(current_url, params=params, headers=custom_headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            product_links = soup.find_all("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")
            
            for product in product_links:
                if len(product_links_list) >= num_products:
                    break
                link = "https://www.amazon.com.mx" + product['href']
                product_links_list.append(link)

            next_page = soup.find("a", class_="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator")
            if next_page and 'href' in next_page.attrs:
                current_url = "https://www.amazon.com.mx" + next_page['href']
            else:
                break
        else:
            print(f"No se pudo acceder a la página para {product_name}. Código de estado:", response.status_code)
            break

    return product_links_list

# Configuración para Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument(f"user-agent={generate_user_agent()}")

# Función para obtener reseñas y calificaciones de Amazon
def get_reviews_amazon(link):
    driver = webdriver.Chrome(options=chrome_options)
    reviews_data = []

    try:
        driver.get(link)
        while True:
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "review-text-content"))
                )
                reviews = driver.find_elements(By.CLASS_NAME, "review-text-content")
                ratings = driver.find_elements(By.CLASS_NAME, "a-icon-alt")

                for review, rating in zip(reviews, ratings):
                    try:
                        stars = float(rating.get_attribute('textContent').split(' ')[0])
                        reviews_data.append((review.text, stars))
                    except ValueError:
                        continue  # En caso de no poder extraer una calificación válida

                try:
                    next_button = driver.find_element(By.XPATH, "//li[@class='a-last']/a")
                    if next_button.is_enabled():
                        next_button.click()
                        time.sleep(2)
                    else:
                        break
                except (NoSuchElementException, ElementClickInterceptedException):
                    break
            except TimeoutException:
                print("Tiempo de espera agotado en la carga de reseñas para:", link)
                break
    finally:
        driver.quit()

    return reviews_data

# Funcion para procesar productos en un rango dado y almacenar reseñas en el DataFrame global
def process_products(start_idx, end_idx):
    global all_reviews
    for i in range(start_idx, end_idx):
        product = productos[i]
        print(f"Extrayendo enlaces de productos para: {product}")
        product_links = get_product_links(product)

        for link in product_links:
            print(f"Extrayendo reseñas del producto: {link}")
            try:
                reviews = get_reviews_amazon(link)
                # Agregar reseñas al DataFrame con exclusion mutua
                with data_lock:
                    for review, rating in reviews:
                        all_reviews = pd.concat([all_reviews, pd.DataFrame({"Producto": [product], "Reseña": [review], "Calificación": [rating]})])
            except Exception as e:
                print(f"Error al extraer reseñas de {link}: {e}")
                continue

# Crear hilos para paralelizar la extraccion
threads = []
for i in range(num_cores):
    start_idx = i * chunk_size
    end_idx = start_idx + chunk_size if i < num_cores - 1 else len(productos)
    thread = Thread(target=process_products, args=(start_idx, end_idx))
    threads.append(thread)
    thread.start()

# Esperar a que todos los hilos terminen
for thread in threads:
    thread.join()

# Guardar el DataFrame en un archivo Excel
all_reviews.to_excel("reseñas_amazon.xlsx", index=False)

print(f"Se han extraído {len(all_reviews)} reseñas y se han guardado en el archivo 'reseñas_amazon.xlsx'.")


Extrayendo enlaces de productos para: Audífonos inalámbricos
Extrayendo enlaces de productos para: Teclado inalámbrico
Extrayendo enlaces de productos para: Aspiradora portátil
Extrayendo enlaces de productos para: Taburete
Extrayendo enlaces de productos para: Cacerola
Extrayendo enlaces de productos para: Cinturón de ejercicio
Extrayendo reseñas del producto: https://www.amazon.com.mx/Ekco-67465-Cacerola-Tapa-Cristal/dp/B09DL95QM3/ref=sr_1_1?dib=eyJ2IjoiMSJ9.9S7l20mC2crHL13EE4ag_mFwcLEAWhWxZixoo420LWogDKP6teKrYfJaLBsUtqk8ARQGPyvhw_QApVs2o0N2DkKAn2ZKPJjX5VXzCQtrge_UL3bwM05wR1m0fknOPSBFrkP7tHH1r1aqFEEhGY_wBmfDa0YWNXgIea8IVmlN2UVVIsYv9gmj9QQTjoVbR2-jKmqpbhUG-mxVUgFFZSZU2NRGVUWhEhgsvDXqeGtD20dJu3J5Pu_vXC4MuYNb7cR9ECarp9lkVSpAKH-GzEtNpl_6uOM1I-Qix7utrTkJh7Q.OCESgDtvxFXTMy-510pnd7yEekpVGfTDgLi3xYWDyjo&dib_tag=se&keywords=Cacerola&qid=1731129614&sr=8-1&ufe=app_do%3Aamzn1.fos.de93fa6a-174c-4df7-be7c-5bc8e9c5a71b
Extrayendo reseñas del producto: https://www.amazon.com.mx/sspa/click?ie=UTF8&sp

C:\Users\Jesus De La Fuente\AppData\Local\Temp\ipykernel_2716\1712669274.py:152: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_reviews = pd.concat([all_reviews, pd.DataFrame({"Producto": [product], "Reseña": [review], "Calificación": [rating]})])


Extrayendo reseñas del producto: https://www.amazon.com.mx/Elite-Fitness-Cinturon-Gimnasio-Levantamiento/dp/B08F456FHM/ref=sr_1_2?dib=eyJ2IjoiMSJ9.xcBrDecWTyTYikksg8u1oVgKQtftZEkAJL5-4X7mYtPEhVtV1fnp8ujTT6KucgPyc7-JIVk1eY4u8lcm6k9jsnLD5yWXP8NK8T58MTrrOIQYl1Qjv6pYQ8fKQK_0ynbUwIWXJg55pOW_0x4_CzDL40MzDmTfmqyDpDgooPXR61eve8kl5B0uVcC6QLFXLxGQCu6IxR21fUpjijmIs-24OxJrl0Ba_aFnUY2n2kxhvAnMGoGXSP1nPDKpHSus51yXUKFdVtVRi6rn_4g740j0LxjJtJ9EvyvwVTSufYOltYo.O0P5QwdESiVQss6QuyYoNCvaMe3dLrzSnD3pFQU_eDM&dib_tag=se&keywords=Cintur%C3%B3n+de+ejercicio&qid=1731129614&sr=8-2&ufe=app_do%3Aamzn1.fos.db4f1a57-87f1-43c5-9a39-0cdca6036b57
Extrayendo reseñas del producto: https://www.amazon.com.mx/Logitech-MK220-Teclado-inal%C3%A1mbricos-espa%C3%B1ol/dp/B0080W1VVC/ref=sr_1_2?dib=eyJ2IjoiMSJ9.LJ-UkBW2zDVJ0XPO_gcgDeOHUstBLc4G-Bl1bpCpJ_kpUE9P9eYaFuoKAEgO1xUaaiFqLljPcg7zOQuLmeGgFekoqgtP2A9dAlY2bJ-mx6k-uVh4PPxezuCTy2M5C9k5zxcugMK1_OT69HhASugAEeo5-sZLTRqzFe0FDVx5nY6rKm8360jFWcQTF0NI5E7wk9CJ5I-lw8zXKyt4Q8sG1DjYSMvCSMedC

# Lista de productos para buscar para un dataset mas variado
productos = [
    "Televisión", "Laptop", "Celular", "Estufa", "Refrigerador",
    "Microondas", "Lavadora", "Secadora", "Auriculares", "Reloj",
    "Monitor", "Impresora", "Cámara", "Tablet", "Proyector",
    "Altavoz", "Router", "Consola", "Teclado", "Mouse",
    "Batería", "Cargador", "Licuadora", "Aspiradora", "Parrilla",
    "Horno", "Cafetera", "Ventilador", "Aire acondicionado", "Plancha",
    "Tostadora", "Deshumidificador", "Termo", "Humidificador", "Lampara",
    "Radiador", "Purificador", "Extractor", "Estéreo", "Secador",
    "Enchufe", "Cable", "Bocina", "Disco duro", "Memoria",
    "Linterna", "Taladro", "Escoba", "Pala", "Escritorio",
    "Silla", "Mesa", "Colchón", "Cojín", "Edredón",
    "Almohada", "Cortina", "Espejo", "Cuadro", "Vela",
    "Jarrón", "Plato", "Taza", "Vaso", "Cubiertos",
    "Sartén", "Olla", "Cuchillo", "Rallador", "Tabla",
    "Colador", "Batidora", "Freidora", "Exprimidor", "Molde",
    "Cepillo", "Peine", "Toalla", "Bolsas", "Rastrillo",
    "Esponja", "Shampoo", "Jabón", "Perfume", "Desodorante",
    "Bicicleta", "Scooter", "Patineta", "Pelota", "Pesas",
    "Colchoneta", "Botella", "Mochila", "Maleta", "Paraguas",
    "Guantes", "Gorra", "Sombrero", "Calcetines", "Reloj"
]